# <a name="top">Exercise 2: HGNC and MySQL</span>

## Table of content
0. <a href="#import_and_preparations">Imports and preparations</a>
0. <a href="#problem">Defining the problem</a>
0. <a href="#objective">Objective</a>
0. <a href="#tasks_in_general">Tasks in general</a>
0. <a href="#tasks">Tasks</a>
0. <a href="#problem">Defining the problem</a>

SQLite often used on mobile phones; for small solutions and for teaching

## <a id="import_and_preparations">Imports and preparations</a>

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import os
import json
import pymysql
import re
base = os.path.join(os.environ['BUG_FREE_EUREKA_BASE'])
%matplotlib inline

In [3]:
pathToFile = os.path.join(base,'data','exercise05','nodes_in_Alzheimer_BEL_disease_network.tsv')
df = pd.read_table(pathToFile)

## <a id="problem">Defining the problem</a>
We have already loaded a list of biological entities in the context of Alzheimer's disease in a pandas `DataFrame`. The column **`value`** identifies in this `DataFrame` biological entities in specific namespace (stored in the column `namespace`). **`namespace`** and **`value`** together are a ***unique identifier*** for the entities. 

One of the most used namespaces in our dataset is HGNC. The HGNC database is maintained by the HUGO  Gene Nomenclature Committee and stores approved gene names, symbols (short-form abbreviation, column our `value`) and addtional information about genes.

OK, even we have now a clear identify with the short name in our `DataFrame`: But do you know the recommended names of these short names, to which gene family they belong or the gene ID from NCBI?

In [4]:
list(df[(df.namespace=='HGNC')].value[:10])

['BACE1',
 'APP',
 'CLSTN1',
 'KLC1',
 'PSEN1',
 'NCSTN',
 'APH1A',
 'PSEN2',
 'ARRB2',
 'GPR3']

 Ice cream for all if you can give me all recommended names (without any further search)!

## <a id="objective">Objective</a>
+ Find new insights into Alzheimer's disease by analysing the information about genes

## <a id="tasks_in_general">Tasks in general</a>

   0. Analyse the JSON file, find a way to automatically scan the whole json for datatypes
   0. Connect to MySQL, create database
   0. Create appropriate tables in MySQL database
   0. Store the data in the database
   0. Find the gene family with the highest frequency 
       + by combining 
           + pandas dataframe
           + MySQL database
       + loading database into dataframe
       + loading dataframe into database
       
links:
+ [Reference manual](http://dev.mysql.com/doc/refman/5.7/en/)
+ [HUGO Gene Nomenclature Committee (HGNC)](http://www.genenames.org/)

## <a id="tasks">Tasks</a>

### Load the JSON file

The data is in the JSON format - this means that all atomic data is in nested dictionaries and lists.

In [15]:
data_path = os.path.join(base,'data', 'exercise02', 'hgnc_complete_set.json')

with open(data_path) as f:
    hgnc_json = json.load(f)

### Exploration of data structure

We want to understand what data is inside this json. To do this, we'll look at the keys of each nested dictionary.

The first set of keys shows that there is a response (the data) and a response header (data about the way it was downloaded). We will further explore the response.

In [6]:
hgnc_json.keys()

dict_keys(['responseHeader', 'response'])

The response contains `numFound`, which lists how many results there are in `docs`. Disregard `start`.

In [50]:
hgnc_json['response'].keys()

dict_keys(['numFound', 'docs', 'start'])

In [11]:
hgnc_json['response']['docs'][0]

{'_version_': 1546503090507612160,
 'ccds_id': ['CCDS12976'],
 'cosmic': 'A1BG',
 'date_approved_reserved': '1989-06-30',
 'date_modified': '2015-07-13',
 'ensembl_gene_id': 'ENSG00000121410',
 'entrez_id': '1',
 'gene_family': ['Immunoglobulin like domain containing'],
 'gene_family_id': [594],
 'hgnc_id': 'HGNC:5',
 'location': '19q13.43',
 'location_sortable': '19q13.43',
 'locus_group': 'protein-coding gene',
 'locus_type': 'gene with protein product',
 'merops': 'I43.950',
 'mgd_id': ['MGI:2152878'],
 'name': 'alpha-1-B glycoprotein',
 'omim_id': [138670],
 'pubmed_id': [2591067],
 'refseq_accession': ['NM_130786'],
 'rgd_id': ['RGD:69417'],
 'status': 'Approved',
 'symbol': 'A1BG',
 'ucsc_id': 'uc002qsd.5',
 'uniprot_ids': ['P04217'],
 'uuid': 'c5fd27c5-7aa4-447c-83b0-1ccc73d90925',
 'vega_id': 'OTTHUMG00000183507'}

`docs` is a list where each entry is the data associated with a gene. Below is an example of the first element of this list:

In [55]:
print(json.dumps(hgnc_json['response']['docs'][0], indent=5))

{
     "locus_type": "gene with protein product",
     "pubmed_id": [
          2591067
     ],
     "date_modified": "2015-07-13",
     "uniprot_ids": [
          "P04217"
     ],
     "status": "Approved",
     "vega_id": "OTTHUMG00000183507",
     "rgd_id": [
          "RGD:69417"
     ],
     "refseq_accession": [
          "NM_130786"
     ],
     "name": "alpha-1-B glycoprotein",
     "gene_family": [
          "Immunoglobulin like domain containing"
     ],
     "ensembl_gene_id": "ENSG00000121410",
     "cosmic": "A1BG",
     "merops": "I43.950",
     "hgnc_id": "HGNC:5",
     "location": "19q13.43",
     "omim_id": [
          138670
     ],
     "uuid": "c5fd27c5-7aa4-447c-83b0-1ccc73d90925",
     "symbol": "A1BG",
     "ccds_id": [
          "CCDS12976"
     ],
     "date_approved_reserved": "1989-06-30",
     "ucsc_id": "uc002qsd.5",
     "location_sortable": "19q13.43",
     "_version_": 1546503090507612160,
     "gene_family_id": [
          594
     ],
     "locus_group"

### Create a Database Schema

Analyze the structure of each entry. Depending on the data type, decide what sort of relation you need to store the data. For example, a list would correspond to a 1-to-many relationship, while an atomic would correspond to a 1-to-1.

Your goal is to connect to your database, build an appropriate schema, and upload the data from the hgnc_json file. 

Choose one 1-to-n relationship and some 1-to-1 for your schema.

In [5]:
secrets_path = os.path.join(base, 'secrets.json')

with open(secrets_path) as f:
    secrets = json.load(f)
    
# for mysql you have a port: 3306

In [6]:
pd.io.sql._SQL_TYPES['string']['mysql']='VARCHAR (255)'
# fixes the problem of too short varchar fields

# if you find a problem in a specific library in python,
# there are often solutions to solve it,
# whether you have precompiled languages
# 

In [58]:
db_params = secrets['test_db']

conn = pymysql.connect(**db_params)
# ** => splat

In [59]:
schema_sql = """
use hgnc
"""

with conn.cursor(pymysql.cursors.DictCursor) as cursor:
    cursor.execute(schema_sql)
    print(*cursor.fetchone())

TypeError: print() argument after * must be an iterable, not NoneType

## ToDos
+ create a database with the name `hgnc`
+ Analyse you data structure by iterating over the data and identify 
    + datatypes
    + relationship type (1 to 1, 1 to many)
+ use a new cursor
+ follow the suggestions below
+ Upload the Data

## Solution

## Create the database 

In [10]:
conn = pymysql.connect(**secrets['test_db'])
cursor = conn.cursor(pymysql.cursors.DictCursor)

In [8]:
schema_sql = """
create schema if not exists hgnc
"""

with conn.cursor(pymysql.cursors.DictCursor) as cursor:
    cursor.execute(schema_sql)
    print(cursor.fetchone())

/Users/alex/anaconda/lib/python3.5/site-packages/pymysql/cursors.py:323: Warning: (1007, "Can't create database 'hgnc'; database exists")
  self._do_get_result()


1

None


In [12]:
# cursor.execute('use hgnc')
cursor.execute('show tables')
cursor.fetchall()

1

[{'Tables_in_hgnc': 'gene'}]

### Analyse you data structure by iterating over the data

In [16]:
docs = hgnc_json['response']['docs']
print(type(docs))

<class 'list'>


#### Analysis of hgnc_json data types:
As we can see, there is no mixed types:

In [17]:
dataTypes = {}
for entry in docs:
    for key, value in entry.items():
        if key not in dataTypes:
            dataTypes[key] = [type(value)]
            if isinstance(value, list):
                dataTypes[key].append(type(value[0]))
        elif type(value) not in dataTypes[key]:
            dataTypes[key].append(type(value))
dataTypes

{'_version_': [int],
 'alias_name': [list, str],
 'alias_symbol': [list, str],
 'bioparadigms_slc': [str],
 'ccds_id': [list, str],
 'cd': [str],
 'cosmic': [str],
 'date_approved_reserved': [str],
 'date_modified': [str],
 'date_name_changed': [str],
 'date_symbol_changed': [str],
 'ena': [list, str],
 'ensembl_gene_id': [str],
 'entrez_id': [str],
 'enzyme_id': [list, str],
 'gene_family': [list, str],
 'gene_family_id': [list, int],
 'hgnc_id': [str],
 'homeodb': [int],
 'horde_id': [str],
 'imgt': [str],
 'intermediate_filament_db': [str],
 'iuphar': [str],
 'kznf_gene_catalog': [int],
 'lncrnadb': [str],
 'location': [str],
 'location_sortable': [str],
 'locus_group': [str],
 'locus_type': [str],
 'lsdb': [list, str],
 'mamit-trnadb': [int],
 'merops': [str],
 'mgd_id': [list, str],
 'mirbase': [str],
 'name': [str],
 'omim_id': [list, int],
 'orphanet': [int],
 'prev_name': [list, str],
 'prev_symbol': [list, str],
 'pseudogene.org': [str],
 'pubmed_id': [list, int],
 'refseq_acc

#### Same, done with set, but it works slower

In [88]:
from collections import defaultdict
dataTypes = defaultdict(set)
# ... write here your code to fill up the dataTypes dictionary like
# dataType = {'column1':[int],'column2':[str,int],...}

for doc in docs:
    for key, value in doc.items():
        if isinstance(value, list):
            dataTypes[key].add(type(value))
            [dataTypes[key].add(type(value[i])) for i in range(len(value))]
        else:
            dataTypes[key].add(type(value))
dataTypes

defaultdict(set,
            {'_version_': {int},
             'alias_name': {str, list},
             'alias_symbol': {str, list},
             'bioparadigms_slc': {str},
             'ccds_id': {str, list},
             'cd': {str},
             'cosmic': {str},
             'date_approved_reserved': {str},
             'date_modified': {str},
             'date_name_changed': {str},
             'date_symbol_changed': {str},
             'ena': {str, list},
             'ensembl_gene_id': {str},
             'entrez_id': {str},
             'enzyme_id': {str, list},
             'gene_family': {str, list},
             'gene_family_id': {int, list},
             'hgnc_id': {str},
             'homeodb': {int},
             'horde_id': {str},
             'imgt': {str},
             'intermediate_filament_db': {str},
             'iuphar': {str},
             'kznf_gene_catalog': {int},
             'lncrnadb': {str},
             'location': {str},
             'location_sortable': 

Now we check if there could be more than one datatype per key. We can do this by iterating over the dictionary `dataTypes`. This time we have curly brackets surronding the list compression. This create a set.

In [67]:
my_lst = [1,1,1,1,12,2,2,3,4,54,5,6]
{i for i in my_lst if i<3}
# now you have a set!

{1, 2}

In [ ]:
# Now deside if we have perhaps mixed datatypes in your columns. That could be a problem

In [ ]:
# create a list of columns which can be used for the creation
# of the main table because they are 1-to-1 and 
# a list of columns with more than value 

In [89]:
lst_1_to_1 = [key for key, value in dataTypes.items() if len(value) == 1]
lst_1_to_many = [key for key, value in dataTypes.items() if len(value) == 2]

In [91]:
lst_1_to_1
lst_1_to_many

['locus_type',
 'date_modified',
 'snornabase',
 'location_sortable',
 'homeodb',
 'date_symbol_changed',
 'horde_id',
 'intermediate_filament_db',
 'merops',
 'bioparadigms_slc',
 'date_approved_reserved',
 'ucsc_id',
 'mamit-trnadb',
 '_version_',
 'locus_group',
 'imgt',
 'pseudogene.org',
 'vega_id',
 'status',
 'name',
 'location',
 'iuphar',
 'ensembl_gene_id',
 'cosmic',
 'hgnc_id',
 'uuid',
 'kznf_gene_catalog',
 'symbol',
 'cd',
 'mirbase',
 'date_name_changed',
 'orphanet',
 'lncrnadb',
 'entrez_id']

['pubmed_id',
 'alias_symbol',
 'uniprot_ids',
 'ena',
 'refseq_accession',
 'gene_family',
 'omim_id',
 'prev_symbol',
 'lsdb',
 'enzyme_id',
 'gene_family_id',
 'alias_name',
 'rgd_id',
 'prev_name',
 'ccds_id',
 'mgd_id']

Let's create automatically the main table `gene` with `to_sql` method of `pandas.DataFrame`. Exclude the columns which

In [ ]:
# create on the fly and try to put it in db? or something like that

In [ ]:
# load the json into a DataFrame as we did it before
# 1. all columns in the table named 'gene'
# 2. create dynamically first DataFrames and then
#    tables named as the columns with 1-to-many relation. 
#    Store also the foreign key in this DataFrame 

In [ ]:
# Code 1

In [17]:
df_hgnc = pd.DataFrame(docs)

In [ ]:
kwargs = {
    
}

In [ ]:
regex_not_allwed = re.compile["[^0-9a-zA-Z$]"]

Check this MySQL website [Schema Object Names](http://dev.mysql.com/doc/refman/5.7/en/identifiers.html) for valid column names.

```
[^0-9a-zA-Z$_] are allowed characters
```

In [ ]:
# correct column names with regex

Let's rename our columns in the dataframe

### 1-to1

### 1-to-many